<a href="https://colab.research.google.com/github/mscussiatto/yolo-grow/blob/main/yolov8n-seg-masks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Drive and Time





In [ ]:
!apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8
!export LANG=en_US.UTF-8
!locale -a  # Make sure en_US.UTF-8 is listed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2. Masks to polygons
Create "labels" directory in the root of the job folder (CVATs job dataset).



In [ ]:
import os
import cv2

input_dir = '/content/drive/MyDrive/grow/job_372223-2023_10_27_08_41_57-segmentationmask1.1/SegmentationClass'
output_dir = '/content/drive/MyDrive/grow/job_372223-2023_10_27_08_41_57-segmentationmask1.1/labels'

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()




# 3. Train YOLOv8-seg
Train semantic segmentation model using custom dataset.


In [ ]:
!pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')  # load a pretrained model (recommended for training)

model.train(data='/content/drive/MyDrive/grow/config.yaml', epochs=200, imgsz=640)

#4. Running Inference and saving results/crops
Trying to get the RGB mask in the crops.

In [ ]:
!pip install ultralytics

##Normal CLI saving crops and inference results.

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('/content/drive/MyDrive/grow/models/yolov8n-seg_200e_640p/weights/best.pt')

# Run inference on 'bus.jpg' with arguments
model.predict('/content/drive/MyDrive/grow/dataset_1/images/test', save=True, save_crop=True, imgsz=640, conf=0.8)

#5. Get all the binary masks saved in one image and individually. same as in binary_mask.py
Runs inference, save results and save individual binary masks per inference.

In [ ]:
from ultralytics import YOLO
import cv2
import torch
from pathlib import Path

# Load a pretrained YOLOv8n-seg Segment model
model = YOLO('/content/drive/MyDrive/grow/models/yolov8n-seg_200e_640p/weights/best.pt')

# Run inference on an image
results = model('/content/drive/MyDrive/grow/dataset_1/images/test/2023-10-27_05-54-40.jpeg')  # results list

result = results[0]

print(result.names)
# print(result.boxes.xyxy)
# print(result.boxes.conf)
# print(result.boxes.cls)
# print(result.masks.data)

Path("./output/").mkdir(parents=True, exist_ok=True)

cv2.imwrite(f"./output/original_image.jpg", result.orig_img)

seg_classes = list(result.names.values())
# seg_classes = ["salanova_euler", "seedling", "empty", "radicchio_trevigiano"]

for result in results:

    masks = result.masks.data
    boxes = result.boxes.data

    clss = boxes[:, 5]
    print("clss")
    print(clss)

    #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
    obj_indices = torch.where(clss != -1)
    obj_masks = masks[obj_indices]
    obj_mask = torch.any(obj_masks, dim=0).int() * 255
    cv2.imwrite(str(f'./output/all-masks.jpg'), obj_mask.cpu().numpy())

    #MASK OF ALL INSTANCES OF A CLASS
    for i, seg_class in enumerate(seg_classes):

        obj_indices = torch.where(clss == i)
        print("obj_indices")
        print(obj_indices)
        obj_masks = masks[obj_indices]
        obj_mask = torch.any(obj_masks, dim=0).int() * 255

        cv2.imwrite(str(f'./output/{seg_class}s.jpg'), obj_mask.cpu().numpy())

        #MASK FOR EACH INSTANCE OF A CLASS
        for i, obj_index in enumerate(obj_indices[0].cpu().numpy()):
            obj_masks = masks[torch.tensor([obj_index])]
            obj_mask = torch.any(obj_masks, dim=0).int() * 255
            cv2.imwrite(str(f'./output/{seg_class}_{i}.jpg'), obj_mask.cpu().numpy())